<a href="https://colab.research.google.com/github/notShyam7/realtime-hand-gesture-recognition/blob/main/HandGesture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d gti-upm/leapgestrecog

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!unzip leapgestrecog.zip

In [ ]:
!pip install opencv-python tensorflow numpy

In [ ]:
# colab mount
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Define your dataset directories
dataset_directories = ["/content/leapGestRecog", "/content/leapgestrecog"]

In [ ]:
# Function to load and preprocess the dataset from multiple directories
def load_and_preprocess_data(dataset_directories):
    images = []
    labels = []

    for dataset_directory in dataset_directories:
        for class_folder in os.listdir(dataset_directory):
            class_dir = os.path.join(dataset_directory, class_folder)

            if os.path.isdir(class_dir):
                for gesture_folder in os.listdir(class_dir):
                    gesture_dir = os.path.join(class_dir, gesture_folder)

                    if os.path.isdir(gesture_dir):
                        for image_file in os.listdir(gesture_dir):
                            if image_file.endswith(".png"):
                                # Load the image
                                img = cv2.imread(os.path.join(gesture_dir, image_file))
                                img = cv2.resize(img, (64, 64))  # Resize to a consistent size
                                img = img / 255.0  # Normalize pixel values
                                images.append(img)

                                # Extract class label from the class folder name (e.g., "00")
                                label = int(class_folder)
                                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [ ]:
# Load and preprocess the dataset from both directories
train_images, train_labels = load_and_preprocess_data(dataset_directories)

In [ ]:
# Split the data into training and testing sets
test_size = 0.2  # Adjust the ratio as needed
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=test_size, random_state=42)

In [ ]:
# Verify the shapes of the resulting datasets
print("Number of training images:", len(train_images))
print("Number of training labels:", len(train_labels))
print("Number of testing images:", len(test_images))
print("Number of testing labels:", len(test_labels))

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Determine the number of unique labels in the training data
num_classes = len(np.unique(train_labels))

In [ ]:
# Convert labels to one-hot encoded format
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Define your CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Use num_classes here
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

In [ ]:
model.save('/content/gdrive/MyDrive/handgesture/gestures.h5')

In [ ]:
import os
print(os.path.abspath('Desktop/handgesture/gestures.h5'))

/Users/ghanshyam/Desktop/handgesture/Desktop/handgesture/gestures.h5


In [ ]:
import tensorflow as tf

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model('/Users/ghanshyam/Desktop/handgesture/gestures.h5')  # Update with the path to your model file

# Define a function to preprocess the frame for model input
def preprocess_frame(frame):
    frame = cv2.resize(frame, (64, 64))
    frame = frame / 255.0  # Normalize pixel values
    return frame

# OpenCV VideoCapture
cap = cv2.VideoCapture(1)

while True:
    try:
        ret, frame = cap.read()
        if not ret:
            continue

        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame)

        # Make predictions
        predictions = model.predict(np.expand_dims(preprocessed_frame, axis=0))

        # Find the predicted gesture
        predicted_gesture = np.argmax(predictions)

        # Display the recognized gesture on the frame
        cv2.putText(frame, f'Gesture: {predicted_gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame in a window
        cv2.imshow('Hand Gesture Recognition', frame)

        # Break the loop when the 'q' key is pressed
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    except KeyboardInterrupt:
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 11ms/step
